In [ ]:
# Parameters
attr_path = "/srv/stemly/data/7/index/3/attributes/attributes.csv"
index_path = "/srv/stemly/data/7/index/3/index/index.csv"
committed_files = ["/srv/stemly/data/7/index/3/timeseries/993/commit/ts.0/13.1661315396541.parquet",
                   "/srv/stemly/data/7/index/3/timeseries/993/commit/ts.0/13.1662444054358.parquet",
                   "/srv/stemly/data/7/index/3/timeseries/993/commit/ts.0/13.1662553683649.parquet"]
predict_ts_path = "/srv/stemly/data/7/index/3/timeseries/1434/staged/model/16/ts.predict.latest.parquet"
test_ts_path = "/srv/stemly/data/7/index/3/timeseries/1434/staged/model/16/ts.test.latest.parquet"
validate_ts_path = "/srv/stemly/data/7/index/3/timeseries/1434/staged/model/16/ts.validate.latest.parquet"
model = "custom"
model_file_id = "32"
model_resolution = None
reference_resolution = "MS"
project_offset = 1
project_periods = 18
project_resolution = "MS"
ts_train_start = "2020-07-01 00:00:00"
ts_train_v_end = "2022-05-01 00:00:00"
ts_train_t_end = "2022-04-01 00:00:00"
ts_train_p_end = "2022-06-01 00:00:00"
ts_test_start = "2022-04-01 00:00:00"
ts_test_end = "2022-05-01 00:00:00"
ts_test_duration = 1
ts_validate_start = "2022-05-01 00:00:00"
ts_validate_end = "2022-06-01 00:00:00"
ts_validate_duration = 1
ts_predict_start = "2022-06-01 00:00:00"
ts_predict_end = "2023-12-01 00:00:00"
ts_predict_duration = 18
redis_url = "redis://127.0.0.1:6300"
job_id = "6bf40e7f-aced-40f7-86fe-f49f8d23b343"
job_user_id = "3"
resource_id = "16"
resource_version = 1661403479559
resource_type = "modelbase"
workspace_id = "7"
job_name = "custom"
job_triggered_at = "2022-08-25 04:58:00"
job_triggered_by = "Mallikarjuna Reddy Padigapati"


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm as tqdm 
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)

In [ ]:
workspace = 'w' + workspace_id
verbose = True

In [ ]:
main_group_index = ['sku', 'customer', 'warehouse']
mat_group = 'sku'
freq = "MS"
load_data_from_local = False
export_data = True

In [ ]:
price_index = 1103

In [ ]:
if load_data_from_local :
    price_var = 'price'
    volume_var = 'dc_offtake'
    main_group_index = ['sku', 'customer', 'warehouse']
    mat_group = 'sku'
else :
    price_var = 'y'
    volume_var = 'y'
    main_group_index = ['id']
    mat_group = 'sku'
    
joining_keys = ["ts"] + main_group_index
joining_keys

## Load data

In [ ]:
if load_data_from_local :
    att_df = pd.read_csv("../../Cleaned_Input/all_att_data.csv")
    vol_df = pd.read_parquet("../../Cleaned_Input/UI_Files/si_monthly_volume.parquet")
    price_df = pd.read_parquet("../../Cleaned_Input/UI_Files/si_monthly_price.parquet")
    list_price_df = pd.read_parquet("../../Cleaned_Input/UI_Files/si_monthly_price.parquet")
    
    vol_df.rename(columns={'date':'ts'},inplace=True)
    price_df.rename(columns={'date':'ts'},inplace=True)
    list_price_df.rename(columns={'date':'ts'},inplace=True)

if not load_data_from_local:
    path_index_val = os.listdir("/srv/stemly/data/{}/index".format(workspace_id))[0]
    path_index = "/srv/stemly/data/{}/index".format(workspace_id) + "/" + str(path_index_val)
    vol_df = pd.concat([pd.read_parquet(fp) for fp in committed_files])

    att_df = pd.read_csv(attr_path)
    price_parquet_file_path = path_index + "/timeseries/"+ str(price_index)+ "/commit/ts.0"
    price_file_names =  os.listdir(price_parquet_file_path)
    price_file_paths = [price_parquet_file_path+ "/" + i for i in price_file_names]
    print(price_file_paths)

    price_df = pd.concat([pd.read_parquet(fp) for fp in price_file_paths])


In [ ]:
sub_group_index = ['ts','sku','customer']
att_df_sub = att_df[[ 'sku', 'customer', 'item_type', 'material', 'ph_level2', 'ph_level3',
       'sku_description', 'cases','uom']].drop_duplicates()


In [ ]:
vol_df.rename(columns={volume_var:'y'},inplace=True)
price_df.rename(columns={price_var:'price'},inplace=True)

price_df = price_df[main_group_index+['price','ts']]
vol_df = vol_df[main_group_index+['y','ts']]

vol_df = vol_df.groupby(joining_keys).sum().reset_index()
price_df = price_df.groupby(joining_keys).mean().reset_index()
if not load_data_from_local:
    att_df.rename(columns={'__idx':'id'},inplace=True)

print(vol_df.shape)
df_base = pd.merge(vol_df, price_df, on =joining_keys, how='left')
df_base = pd.merge(df_base, att_df,  on = main_group_index, how='left')
df_base
df_base.loc[df_base['y'] < 0,'y'] = 0
print(df_base.shape)

df_sub = df_base[['ts','sku','customer','y','price']]
df_sub1 = df_sub.groupby(sub_group_index).agg({'y':'sum','price':'mean'}).reset_index()
df_sub1 = pd.merge(df_sub1,att_df_sub,on=['sku','customer'],how='left')
df_sub1['price'].replace([np.inf,0, -np.inf],np.nan,inplace=True)
df_sub1['price'] = df_sub1.groupby(sub_group_index)['price'].transform(lambda x: x.ffill())
df_sub1['price'] = df_sub1.groupby(sub_group_index)['price'].transform(lambda x: x.bfill())
df_sub1['price'].replace([np.nan],0,inplace=True)

df_main = df_sub1.copy()
df_sub1.head()


In [ ]:
data = df_main.copy()
data.head()

unique_id = data[['sku','customer']].drop_duplicates().reset_index(drop=True)
unique_id['id'] = unique_id.index
data = pd.merge(data,unique_id,on=['sku','customer'])
data.head()



def to_days(x):
    switcher = {0: "Mon", 1: "Tue", 2: "Wed", 3: "Thu", 4: "Fri", 5: "Sat", 6: "Sun"}
    return switcher.get(x)
def week_of_month(date_value):
    return (date_value.isocalendar()[1] - date_value.replace(day=1).isocalendar()[1] + 1)
data["month"] = data["ts"].dt.month 
data["year"] = data["ts"].dt.year 

print(data.shape)
print(data.columns)
id_sales_train = data[data['ts'] <= ts_predict_start]
id_sales_train = id_sales_train.groupby('id')['y'].sum().reset_index()
id_with_sales_df = id_sales_train[id_sales_train['y'] > 0]
id_with_sales = id_with_sales_df.id.unique()
data = data[(data['id'].isin(id_with_sales))]
print(data.shape)
data.head()

## Helper

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

def r2_rmse(g, actual="y", prediction="yhat"):
    r2 = r2_score( g[actual], g[prediction] )
    rmse = np.sqrt( mean_squared_error( g[actual], g[prediction] ) )
    actuals = g[actual].sum()
    predicted = g[prediction].sum()
    return pd.Series( dict(  actuals = actuals,prediction = predicted,r2 = r2, rmse = rmse) )

def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

def mean_absolute_percentage_error(g, actual='y', prediction='yhat'): 
    y_true, y_pred = np.array(g[actual]), np.array(g[prediction])
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mase(g, actual='y', prediction='yhat'):
    """
    Computes the MEAN-ABSOLUTE SCALED ERROR forcast error for univariate time series prediction.
    
    See "Another look at measures of forecast accuracy", Rob J Hyndman
    
    parameters:
        training_series: the series used to train the model, 1d numpy array
        testing_series: the test series to predict, 1d numpy array or float
        prediction_series: the prediction of testing_series, 1d numpy array (same size as testing_series) or float
        absolute: "squares" to use sum of squares and root the result, "absolute" to use absolute values.
    
    """
    n = g.shape[0]
    d = np.abs(  np.diff( training_series) ).sum()/(n-1)
    
    errors = np.abs(testing_series - prediction_series )
    return errors.mean()/d

def ape(g, actual='y', prediction='yhat'):
    ape_vals = abs(g[actual] - g[prediction]) / g[actual]
    ape_vals = ape_vals*100
    ape_vals = ape_vals.round(2)  
    return ape_vals

def extract_sliding(df, window=8, gap=4, target="y"):
    lags = [x for x in range(gap + 1, gap + window + 1)]
    aa = df[["id", "ts", "y"]].assign(**{
        "{}_lag_{}".format(col, l): df.groupby(["id"], observed=True)[col].transform(lambda x: x.shift(l))
        for l in lags
        for col in [target]
    })
    aa.sort_values(by=["id", "ts"], inplace=True)
    aa.reset_index(inplace=True)
    aa.rename(columns={"index": "sub_id"}, inplace=True)

    b = aa.drop(columns=["id", "ts", "y"]).melt(
        id_vars="sub_id",
        value_name="y",
        var_name="lags"
    )
    b["lags"] = [int(x.replace("y_lag_", "")) for x in b["lags"]]
    return aa, b

def calc_metrics(y, yhat):
    """
    Calculate metrics for a single series.
    """
    # mae
    mae = np.mean(np.abs(y - yhat))

    # smape
    smape = 2 * np.abs((y - yhat) / (y + yhat))
    smape = np.nan_to_num(smape, 0)
    smape = np.mean(smape)

    # mape
    mape = np.abs((y - yhat) / y)
    mape = np.nan_to_num(mape, 0)
    mape = np.mean(mape)

    # maape
    maape = np.arctan2(np.abs(y - yhat), np.abs(y))
    maape = np.mean(maape)

    # rmse
    rmse = (y - yhat) * (y - yhat)
    rmse = np.sqrt(np.mean(rmse))

    return {
        "rmse": rmse,
        "mae": mae,
        "smape": smape,
        "mape": mape,
        "maape": maape
    }

## Data

### Processed

In [ ]:
freq = "MS"

In [ ]:
df = data.copy()
# df["year"] = df.ts.dt.year
df["month"] = df.ts.dt.month
df.dropna(subset=["y"], inplace=True)
df.reset_index(drop=True, inplace=True)
df = df[ df['customer'] == 'TMALL']

In [ ]:
df.head()

In [ ]:
from dateutil.relativedelta import relativedelta
def generate_dates_new(
        last_test_date="2021-12-27",
        gap_in_days=28,
        test_horizon_in_days=28,
        version_skip = 28,
        fc_horizon_in_days=84,
        freq="W-Mon",
        n_rounds=2
):
    ts_test_end = pd.to_datetime(last_test_date) 
    ts_test_start = ts_test_end - pd.Timedelta(days=test_horizon_in_days - 1)
   
    ts_predict_start = ts_test_end + pd.Timedelta(days=1)
    ts_predict_end = ts_predict_start + pd.Timedelta(days=fc_horizon_in_days-1)

   
    test_dates = pd.date_range(
        start=ts_test_start,
        end=ts_test_end,
        freq=freq,
    )
    forecast_dates = pd.date_range(
        start=ts_predict_start,
        end=ts_predict_end,
        freq=freq,
    )
    all_dates1 = df.ts.unique()
    train_dates = all_dates1[all_dates1 < np.min(test_dates)]
    a = test_dates[0].month
    all_dates_new = {
        a: {
            
            "train" : pd.to_datetime(train_dates),
            "test": test_dates,
            "forecast": forecast_dates,
        },
    }
    for k in range(n_rounds):
        a = a - 1
        if a == 0:
            a = 12
        train_dates = pd.date_range(
            start=pd.to_datetime(train_dates[0]),
            end=pd.to_datetime(train_dates[-1]) - relativedelta(months=version_skip),
            freq=freq
        )
        test_dates = pd.date_range(
            start=test_dates[0]- relativedelta(months=version_skip),
            end=test_dates[-1] - relativedelta(months=version_skip),
            freq=freq
        )
        forecast_dates = pd.date_range(
            start=forecast_dates[0]- relativedelta(months=version_skip),
            end=forecast_dates[-1] - relativedelta(months=version_skip),
            freq=freq
        )
        all_dates_new.update({
            a: {
                "train" : train_dates,
                "test": test_dates,
                "forecast": forecast_dates,
            }
        })
    return all_dates_new


In [ ]:
ts_test_end = "2022-08-01 00:00:00"
all_dates = generate_dates_new(
#     last_test_date="2021-12-27",
    last_test_date= ts_test_end,
    freq=freq,
    gap_in_days=30,
    version_skip = 1,
    fc_horizon_in_days = 367,
    test_horizon_in_days=30,
    n_rounds=2
)
all_dates

versions = []
for i in all_dates.keys():
    versions.append(i)
m=list(all_dates.keys())[0]
    
versions ,all_dates 

In [ ]:
for a in df.sku.unique():
    ts = df.loc[df.sku == a].groupby("ts").sum()
    plt.figure(figsize=(15,5))
    plt.plot(ts.y)
    plt.title(a)
    plt.show()

In [ ]:
for a in df.sku.unique():
    ts = df.loc[df.sku == a].groupby("ts").sum()
    a = ts.loc[ts.index < "2022-06-01"]
    b = ts.loc[ts.index >= "2022-06-01"]
    plt.figure(figsize=(15,5))
    plt.bar(a.index, a.y, width=5)
    plt.bar(b.index, b.y, width=5)
    plt.show()

### Differenced

In [ ]:
df_diff = []
all_ids = df.id.unique()
initial = []
for i in range(len(all_ids)):
    id_ = all_ids[i]
    tmp = df.loc[df.id == id_].reset_index(drop=True)
    tmp2 = tmp.copy()
    initial.append(tmp["y"][0])
    
    tmp2["y"] = tmp["y"].diff()
    #tmp2["y"].fillna(tmp["y"], inplace=True)
    tmp2.dropna(subset="y", inplace=True)
    df_diff.append(tmp2)
initial_y = pd.DataFrame({"id": all_ids, "initial": initial})
initial_y.set_index("id", inplace=True)
df_diff = pd.concat(df_diff).reset_index(drop=True)

In [ ]:
for a in df_diff.id.unique():
    ts = df_diff.loc[df_diff.id == a].groupby("ts").sum()
    plt.figure(figsize=(15,5))
    plt.plot(ts.y)
    plt.show()

### Attributes

In [ ]:
df.head()

In [ ]:
att_cols = ["id", "sku_description", "sku", "item_type", "uom", "cases", "ph_level2"]
att = df[att_cols].drop_duplicates()
att.reset_index(inplace=True, drop=True)
print("Att", att.shape)

## Dates

## HW-ETS

In [ ]:
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

In [ ]:
df.shape[0]

In [ ]:
def ets_forecast(
    series,
    train_dates,
    test_dates,
):    
    if len(series) > 0:
        # time series must not be empty!

        # fit parameters on val
        train = series.loc[series.index.isin(train_dates)]
        test = series.loc[series.index.isin(test_dates)]
            
        if np.sum(train) == 0:
            # special case when all the values are 0
            out = np.zeros(len(train)+len(test))
            
            return out
        if len(train) == 0:
            # special case when all the values are 0
            out = np.zeros(len(train)+len(test))
            
            return out

        # do cross validation
        configs = [True, False]
        error = ["add"]
        trend = ["add"]
        damped_trend = [True, False]
        seasonal = ["add"]
        seasonal_periods = [4,6,12]
        best_config = None
        best_metric = np.inf
        best_quantile = None
        best_model = None
        for q in [0.95, 0.96, 0.97, 0.98, 0.99]:
            for e in error:
                for t in trend:
                    for d in damped_trend:
                        for s in seasonal:
                            if s is None:
                                model = ETSModel(
                                    np.clip(train, 0, np.quantile(train[np.where(train > 0)[0][0]:], q)),
                                    error=e,
                                    trend=t,
                                    seasonal=s,
                                    damped_trend=d,
                                    seasonal_periods=None,
                                ).fit()
                                
                                yhat = model.predict(0, len(train) - 1)
                                yhat = np.array(yhat)
                                yhat = np.clip(yhat, 0, None)
                                
                                rmse = (train.values - yhat) * (train.values - yhat)
                                rmse = np.sqrt(np.mean(rmse))

                                if rmse < best_metric:
                                    best_metric = rmse
                                    best_model = model
                            else:
                                for ss in seasonal_periods:
                                    try:
                                        model = ETSModel(
                                            np.clip(train, 0, np.quantile(train[np.where(train > 0)[0][0]:], q)),
                                            error=e,
                                            trend=t,
                                            seasonal=s,
                                            damped_trend=d,
                                            seasonal_periods=ss,
                                        ).fit()
                                    except:
                                        model = ETSModel(
                                            np.clip(train, 0, np.quantile(train[np.where(train > 0)[0][0]:], q)),
                                            error=e,
                                            trend=t,
                                            seasonal=None,
                                            damped_trend=d,
                                            seasonal_periods=None,
                                        ).fit()

                                    yhat = model.predict(0, len(train) - 1)
                                    yhat = np.array(yhat)
                                    yhat = np.clip(yhat, 0, None)

                                    rmse = (train.values - yhat) * (train.values - yhat)
                                    rmse = np.sqrt(np.mean(rmse))

                                    if rmse < best_metric:
                                        best_metric = rmse
                                        best_model = model

        # do forecast on train
        yhat_train = best_model.predict(0, len(train) - 1)
        yhat_train = np.array(yhat_train)
        yhat_train = np.clip(yhat_train, 0, None)
        
        # first forecast on test to get a sense of how well the model performs
        yhat_test = best_model.forecast(len(test))
        yhat_test = np.array(yhat_test)
        yhat_test = np.clip(yhat_test, 0, None)
        
        # concatenate all predictions
        out = np.concatenate((yhat_train, yhat_test))

        return out

    return None

In [ ]:
def forecast_ets(
    data,
    dates,
    num_cores=-1,
    freq="MS",
):
    import multiprocessing
    from pandarallel import pandarallel
    
    import warnings
    import logging
    from statsmodels.tools.sm_exceptions import ConvergenceWarning
    
    np.seterr(divide='ignore', invalid='ignore')
    warnings.simplefilter('ignore', ConvergenceWarning)
    logging.getLogger('prophet').setLevel(logging.ERROR)
    
    if num_cores < 0:
        num_cores = multiprocessing.cpu_count()
    else:
        num_cores = min(num_cores, multiprocessing.cpu_count())
        
    pandarallel.initialize(
        nb_workers=num_cores,
        verbose=0,
        progress_bar=True
    )
    
    train_dates = dates["train"]
    test_dates = dates["test"]
    
    s = data.set_index(["id", "ts"], drop=True)["y"]
    s = s.unstack(0).fillna(0)
    s = s.asfreq(freq)
    
    print(train_dates)
    print(test_dates)
    yhat = s.parallel_apply(lambda x: ets_forecast(
        x,
        train_dates,
        test_dates,
    ), axis=0)
    
    

    yhat = yhat.reset_index(drop=True).apply(lambda x: pd.Series(x.dropna().values)).dropna()

    yhat["ts"] = pd.date_range(
        start=max(np.min(train_dates), data.ts.min()),
        end=np.max(test_dates),
        freq=freq
    )
    
    yhat = pd.melt(yhat, id_vars=['ts'], var_name="id", value_name="yhat")
    yhat = pd.merge(df, yhat, on=["ts", "id"], how="left")
    
    yhat = yhat[["id", "ts", "y", "yhat"]+[x for x in yhat.columns if x not in ["id", "ts", "y", "yhat"]]]
          
    return yhat.reset_index(drop=True)

In [ ]:
pred_hws = {}
for i in range(len(versions)):
    pred_hw = forecast_ets(
        df,
        all_dates[versions[i]],
        num_cores=-1,
        freq=freq,
    )
    pred_hws.update({versions[i]: pred_hw})

In [ ]:
yhat_hws = {}
for i in range(len(versions)):
    yhat_hw = pred_hws[versions[i]]
    yhat_hws.update({versions[i]: yhat_hw})

In [ ]:
for i in range(len(versions)):
    yhat_hw = yhat_hws[versions[i]]
    ts = yhat_hw.groupby("ts")[["y", "yhat"]].sum()
    plt.figure(figsize=(15,5))
    plt.plot(ts.y, label="actual")
    plt.plot(ts.yhat, label="forecast")
    plt.title("version {}".format(versions[i]))
    plt.legend()
    plt.show()

In [ ]:
for i in range(len(versions)):
    yhat_hw = yhat_hws[versions[i]]
    ts = yhat_hw.groupby(["sku", "ts"])[["y", "yhat"]].sum()
    ts.reset_index(inplace=True)

    for this_sku in ts.sku.unique():
        _ts = ts.loc[ts.sku == this_sku]
        plt.figure(figsize=(15,5))
        plt.plot(_ts.ts, _ts.y)
        plt.plot(_ts.ts, _ts.yhat)
        plt.title(str(this_sku) + " Version {}".format(versions[i]))
        plt.show()

### Results

In [ ]:
results = []
for i in range(len(versions)):
    pred_hw = pred_hws[versions[i]]
    for _id in pred_hw.id.unique():
        ts = pred_hw.loc[(pred_hw["id"] == _id) ].reset_index(drop=True)
        test = ts.loc[ts.ts.isin(all_dates[versions[i]]['test'])].copy()

        metrics = calc_metrics(test.y, test.yhat)
        metrics.update({
            "id": _id,
            "sku": ts["sku"][0],
            "customer": ts["customer"][0],
            "version" : versions[i],
            "test_start": all_dates[versions[i]]['test'][0],
            "test_end": all_dates[versions[i]]['test'][-1],
            "total_actual": test.y.sum(),
            "total_forecast": test.yhat.sum(),
        })

        results.append(pd.DataFrame(metrics, index=[0]))
results = pd.concat(results).reset_index(drop=True)

main_cols = ["version", "id", "sku", "customer", "test_start", "test_end", "total_actual", "total_forecast"]
results = results[main_cols + list(results.drop(columns=main_cols).columns)].copy()
results["sp1"] = results["total_actual"] / results["total_forecast"]
results["sp1"] = np.nan_to_num(results["sp1"], nan=1, posinf=0, neginf=0)

results = results.sort_values(by=["version","id"]).reset_index(drop=True)
results

In [ ]:
results.mean(), results.median()

## Features

### Lag+Sliding window 

In [ ]:
def sliding_window_features(ddf, gaps=None, target="y", window=3):
    stats = ["mean", "max", "min", "std", "sum", "median"]
    if gaps is None:
        gaps = [1,2,3]
    features = {}
    for gap in gaps:
        df = ddf.copy()
        y = None
        if (target != "y") and (target in df.columns):
            y = df[["id", "ts", "y"]].copy()
            df.drop(columns=["y"], inplace=True)
            df[target].fillna(0, inplace=True)
            df.rename(columns={target: "y"}, inplace=True)
            
        df_lags, df_lags_long = extract_sliding(df, gap=gap, window=window)
       
        df_lags.dropna(inplace=True)
        if y is not None:
            df_lags.rename(columns={"y": target}, inplace=True)
            df_lags = pd.merge(y, df_lags, on=["id", "ts"], how="right")
        
            
        lag_cols = [x for x in df_lags.columns if "_lag" in x]
        sliding_stats = df_lags.copy()
        for i in range(len(lag_cols)):
            a = lag_cols[i]
            for j in range(i+1, len(lag_cols)):
                if j == i:
                    continue
                b = lag_cols[j]
                c = "{}_{}_div_{}".format(target, a.replace("y_", ""), b.replace("y_", ""))
                sliding_stats[c] = sliding_stats[a] / sliding_stats[b]
                sliding_stats[c] = np.nan_to_num(sliding_stats[c], False, 0, 0, 0)
        for s in stats:
            sliding_stats["sliding_{}_{}".format(target, s)] = sliding_stats[lag_cols].agg(s, axis=1)
        sliding_stats["{}_mean_div_median".format(target)] = sliding_stats["sliding_{}_mean".format(target)] / sliding_stats["sliding_{}_median".format(target)]
        sliding_stats["{}_mean_div_sum".format(target)] = sliding_stats["sliding_{}_mean".format(target)] / sliding_stats["sliding_{}_sum".format(target)]
        sliding_stats["{}_median_div_sum".format(target)] = sliding_stats["sliding_{}_median".format(target)] / sliding_stats["sliding_{}_sum".format(target)]
        
        sliding_stats["{}_mean_div_median".format(target)] = np.nan_to_num(sliding_stats["{}_mean_div_median".format(target)], False, 0, 0, 0)
        sliding_stats["{}_mean_div_sum".format(target)] = np.nan_to_num(sliding_stats["{}_mean_div_sum".format(target)], False, 0, 0, 0)
        sliding_stats["{}_median_div_sum".format(target)] = np.nan_to_num(sliding_stats["{}_median_div_sum".format(target)], False, 0, 0, 0)
        
        sliding_stats.sort_values(by=["id", "ts"], inplace=True)
        sliding_stats.reset_index(drop=True, inplace=True)
        sliding_stats.drop(columns=["sub_id"], inplace=True)
        for c in lag_cols:
            sliding_stats.rename(columns={c: c.replace("y", target)}, inplace=True)
        features.update({gap: sliding_stats})
    return features

In [ ]:
y_sliding_all = {}
for i in range(len(versions)):
    this_df = df[df['ts'] < np.min(all_dates[versions[i]]['forecast'])]
    y_sliding = sliding_window_features(this_df, [0,1,2,3])
    y_sliding_all.update({versions[i]: y_sliding})

In [ ]:
tmp = y_sliding_all[m][3]
print(tmp.shape)
tmp.loc[tmp.id == 0]

### Cluster - Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import AgglomerativeClustering

In [ ]:
x_clust_all = {}
for this_version in range(len(versions)):
    this_df = df[df['ts'] < np.min(all_dates[versions[this_version]]['forecast'])]
    x_clust = this_df.pivot_table(index="id", columns="ts", values="y").fillna(0)
    a = MinMaxScaler().fit_transform(x_clust.transpose()).transpose()
    x_clust.loc[:, :] = a
    x_clust_all.update({versions[this_version]: x_clust})

x_clust_all[m].head()

In [ ]:
def plot_dendrogram(model, **kwargs):
    from scipy.cluster.hierarchy import dendrogram
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
n_clusters = 4
clust_model_all = {}
for this_version in range(len(versions)):
    this_df = df[df['ts'] < np.min(all_dates[versions[this_version]]['forecast'])]
    x_clust = x_clust_all[versions[this_version]]
    clust_model = AgglomerativeClustering(
        n_clusters=n_clusters,
        linkage="ward",
        compute_distances=True
    )
    clust_model = clust_model.fit(x_clust)
    clust_model_all.update({versions[this_version]: clust_model})


In [ ]:
# plot the top three levels of the dendrogram
plt.figure(figsize=(15,5))
plot_dendrogram(clust_model_all[m], truncate_mode="level", p=10)
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
all_ids = x_clust_all[m].index
all_ids

In [ ]:
cluster_average_all = {}
for this_version in range(len(versions)):
    this_df = df[df['ts'] < np.min(all_dates[versions[this_version]]['forecast'])]
    y_sliding = sliding_window_features(this_df, [0,1,2,3])
    clust_model = clust_model_all[versions[this_version]]
    x_clust = x_clust_all[versions[this_version]]
    cluster_average = []
    all_ids = x_clust.index
    for i in range(n_clusters):
        idx = np.where(clust_model.labels_==i)
        idx2 = all_ids[idx[0]]

        _at = att.loc[att.id.isin(idx2)]
        ts = x_clust.loc[idx2].values
        t = x_clust.columns
        
        avg = ts.mean(axis=0)
        cluster_average.append(pd.DataFrame({"ts": t, "y": avg, "cluster": i}))

        plt.figure(figsize=(15,5))
        for j in range(ts.shape[0]):
            plt.plot(t, ts[j])
        plt.plot(t, avg, "k", linewidth=5)
        if ts.shape[0] == 1:
            plt.title("Cluster {}: ID: {} Version: {}".format(i, idx[0][0], versions[this_version]))
        else:
            plt.title("Cluster {}: {} Version: {}".format(i, ts.shape[0],  versions[this_version]))
        plt.show()
    cluster_average = pd.concat(cluster_average).reset_index(drop=True)
    cluster_average_all.update({versions[this_version]: cluster_average})

In [ ]:
cluster_features_all = {}
for this_version in range(len(versions)):
    cluster_features = att.copy()
    clust_model = clust_model_all[versions[this_version]]
    cluster_features["cluster_norm"] = clust_model.labels_
    cluster_features = cluster_features[["id", "cluster_norm"]].copy()
    cluster_features_all.update({versions[this_version]: cluster_features})

In [ ]:
clust_avg_yhat_all = {}
for this_version in range(len(versions)):
    cluster_average = cluster_average_all[versions[this_version]]
    clust_avg_yhat = forecast_ets(
        cluster_average.rename(columns={"cluster": "id"}),
        all_dates[versions[this_version]],
        num_cores=-1,
        freq=freq,
    )
    clust_avg_yhat_all.update({versions[this_version]: clust_avg_yhat})

In [ ]:
yhat_clust_all = {}
for this_version in range(len(versions)):
    clust_avg_yhat = clust_avg_yhat_all[versions[this_version]]
    yhat_clust = clust_avg_yhat.copy()
    yhat_clust.rename(columns={"id": "cluster_norm"}, inplace=True)
    yhat_clust_all.update({versions[this_version]: yhat_clust})

In [ ]:
yhat_clust_all[m].cluster_norm.nunique()

In [ ]:

for this_version in range(len(versions)):
    cluster_features = cluster_features_all[versions[this_version]]
    yhat_clust = yhat_clust_all[versions[this_version]]
    cluster_features = pd.merge(
        cluster_features, 
        yhat_clust[["cluster_norm", "ts", "yhat"]],
        on="cluster_norm", how="left"
    )
    cluster_features["cluster_norm"] = cluster_features["cluster_norm"].astype(int)
    cluster_features_all.update({versions[this_version]: cluster_features})
cluster_features_all[m]

### Cluster - Without Scaling

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
x_clust_all = {}
for this_version in range(len(versions)):
    this_df = df[df['ts'] < np.min(all_dates[versions[this_version]]['forecast'])]
    x_clust = this_df.pivot_table(index="id", columns="ts", values="y").fillna(0)
    x_clust_all.update({versions[this_version]: x_clust})

x_clust_all[m].head()

In [ ]:
n_clusters = 4
clust_model_all = {}
for this_version in range(len(versions)):
    this_df = df[df['ts'] < np.min(all_dates[versions[this_version]]['forecast'])]
    x_clust = x_clust_all[versions[this_version]]
    clust_model = AgglomerativeClustering(
        n_clusters=n_clusters,
        linkage="ward",
        compute_distances=True
    )
    clust_model = clust_model.fit(x_clust)
    clust_model_all.update({versions[this_version]: clust_model})


In [ ]:
# plot the top three levels of the dendrogram
plt.figure(figsize=(15,5))
plot_dendrogram(clust_model_all[m], truncate_mode="level", p=10)
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
all_ids = x_clust_all[m].index
all_ids

In [ ]:
cluster_average_all = {}
for this_version in range(len(versions)):
    this_df = df[df['ts'] < np.min(all_dates[versions[this_version]]['forecast'])]
    y_sliding = sliding_window_features(this_df, [0,1,2,3])
    clust_model = clust_model_all[versions[this_version]]
    x_clust = x_clust_all[versions[this_version]]
    cluster_average = []
    all_ids = x_clust.index
    for i in range(n_clusters):
        idx = np.where(clust_model.labels_==i)
        idx2 = all_ids[idx[0]]

        _at = att.loc[att.id.isin(idx2)]
        ts = x_clust.loc[idx2].values
        t = x_clust.columns
        
        avg = ts.mean(axis=0)
        cluster_average.append(pd.DataFrame({"ts": t, "y": avg, "cluster": i}))

        plt.figure(figsize=(15,5))
        for j in range(ts.shape[0]):
            plt.plot(t, ts[j])
        plt.plot(t, avg, "k", linewidth=5)
        if ts.shape[0] == 1:
            plt.title("Cluster {}: ID: {} Version: {}".format(i, idx[0][0], versions[this_version]))
        else:
            plt.title("Cluster {}: {} Version: {}".format(i, ts.shape[0],  versions[this_version]))
        plt.show()
    cluster_average = pd.concat(cluster_average).reset_index(drop=True)
    cluster_average_all.update({versions[this_version]: cluster_average})

In [ ]:
cluster_features_actual_all = {}
for this_version in range(len(versions)):
    cluster_features_actual = att.copy()
    clust_model = clust_model_all[versions[this_version]]
    cluster_features_actual["cluster"] = clust_model.labels_
    cluster_features_actual = cluster_features_actual[["id", "cluster"]].copy()
    cluster_features_actual_all.update({versions[this_version]: cluster_features_actual})

In [ ]:
clust_avg_yhat_all = {}
for this_version in range(len(versions)):
    cluster_average = cluster_average_all[versions[this_version]]
    clust_avg_yhat = forecast_ets(
        cluster_average.rename(columns={"cluster": "id"}),
        all_dates[versions[this_version]],
        num_cores=-1,
        freq=freq,
    )
    clust_avg_yhat_all.update({versions[this_version]: clust_avg_yhat})

In [ ]:
yhat_clust_all = {}
for this_version in range(len(versions)):
    clust_avg_yhat = clust_avg_yhat_all[versions[this_version]]
    yhat_clust = clust_avg_yhat.copy()
    yhat_clust.rename(columns={"id": "cluster"}, inplace=True)
    yhat_clust_all.update({versions[this_version]: yhat_clust})

In [ ]:
yhat_clust_all[m].cluster.nunique()

In [ ]:

for this_version in range(len(versions)):
    cluster_features_actual = cluster_features_actual_all[versions[this_version]]
    yhat_clust = yhat_clust_all[versions[this_version]]
    cluster_features_actual = pd.merge(
        cluster_features_actual, 
        yhat_clust[["cluster", "ts", "yhat"]],
        on="cluster", how="left"
    )
    cluster_features_actual["cluster"] = cluster_features_actual["cluster"].astype(int)
    cluster_features_actual_all.update({versions[this_version]: cluster_features_actual})
cluster_features_actual_all[m]

### Final Features

In [ ]:
def summary_feature(df, dates, target="y", group=None):
    if group is None:
        group = ["id"]
    stats = ["mean", "max", "min", "std", "sum"]
    g = "_".join(group)
    
    train_dates = dates["train"]

    df_features = df.loc[df.ts.isin(train_dates)].copy()
    df_features = df_features.groupby(group)[target].agg(stats)
    df_features.reset_index(inplace=True)
    for col in stats:
        df_features.rename(columns={col: "{}_{}_{}".format(target, g, col)}, inplace=True)
        
    return df_features

In [ ]:
summary_feature(df, all_dates[m], group=["id"])

In [ ]:
aa = y_sliding_all[m][3]
aa[aa['id'] == 2]

In [ ]:
def process_data(
    df, dates,
    yhat_hw, y_sliding, cluster_features_actual, cluster_features,
):
    from scipy import stats
    X = df.copy()
    
    ## encode time

    X["month_sin"] = np.sin(2 * np.pi * X["month"] / 12)
    X["month_cos"] = np.cos(2 * np.pi * X["month"] / 12)
    
    ## summary features at different levels 
    groups = [
        ["id"],
#         ["woy"],
#         ["month"],
#         ["id", "woy"],
#         ["id", "month"],
    ]
    for g in groups:
        sf = summary_feature(X, dates, group=g)
        X = pd.merge(X, sf, on=g, how="left")
        
    ## holt winters projection
    
    
    X = pd.merge(
        X, yhat_hw[["id", "ts", "yhat"]].rename(columns={"yhat": "y_hw"}), 
        on=["id", "ts"], how="left"
    )    
    X["next_hw"] = X[["ts", "id", "y_hw"]].sort_values(by=["id", "ts"]).groupby(["id"])["y_hw"].shift(-1)
    X["prev_hw"] = X[["ts", "id", "y_hw"]].sort_values(by=["id", "ts"]).groupby(["id"])["y_hw"].shift(1)
    X["next_hw"] = X["next_hw"].fillna(X["y_hw"])
    X["prev_hw"] = X["prev_hw"].fillna(X["y_hw"])
    X["slope_hw"] = ((X["y_hw"] - X["prev_hw"]) + ((X["next_hw"] - X["prev_hw"]) / 2)) / 2
    
    ## differenced of Holt Winters
    a = yhat_hw[["id", "ts", "yhat"]].rename(columns={"yhat": "d1_y_hw"}).copy()
    X_diff = []
    all_ids = X.id.unique()
    for i in range(len(all_ids)):
        id_ = all_ids[i]
        tmp = a.loc[a.id == id_].reset_index(drop=True)
        tmp2 = tmp.copy()
        tmp2["d1_y_hw"] = tmp["d1_y_hw"].diff()
        tmp2["d1_y_hw"].fillna(tmp["d1_y_hw"], inplace=True)
        X_diff.append(tmp2)
    X_diff = pd.concat(X_diff).reset_index(drop=True)
    X = pd.merge(
        X, X_diff[["id", "ts", "d1_y_hw"]], 
        on=["id", "ts"], how="left"
    )
    print("base")
    print(X.id.unique())
    #print(X.groupby('id')['ts'].min())
    ## sliding windows of y
    y_sliding['id'] = y_sliding['id'].astype('int64')
    X = pd.merge(X, y_sliding.drop(columns=["y"]), on=["ts", "id"], how="right")    
    print("joined sliding")
    print(X.id.unique())
    #print(X.groupby('id')['ts'].min())
    ## slope estimate of y 
    lag_cols = [x for x in X.columns if ("y_lag_" in x) and ("div" not in x)]
    mid = len(lag_cols) / 2
    mid = lag_cols[int(mid)]
    start = lag_cols[0]
    end = lag_cols[len(lag_cols) - 1]
    X["slope_y"] = ((X[mid] - X[start]) + ((X[end] - X[mid]) / 2)) / 2
    
    ## ratio of current projection to past lags
    for c in lag_cols:
        col = "ratio_hw_to_{}".format(c.replace("y_", ""))
        X[col] = X["y_hw"] / X[c]
        X[col] = np.nan_to_num(X[col], nan=0, posinf=0, neginf=0)
    
    ## cluster features
    X = pd.merge(
        X, cluster_features_actual.rename(columns={"yhat": "y_clust_avg"}), 
        on=["id", "ts"], how="left"
    )
    print("joined cluster feature actual")
    print(X.id.unique())
    #print(X.groupby('id')['ts'].min())
    X["next_clust"] = X[["ts", "id", "y_clust_avg"]].sort_values(by=["id", "ts"]).groupby(["id"])["y_clust_avg"].shift(-1)
    X["prev_clust"] = X[["ts", "id", "y_clust_avg"]].sort_values(by=["id", "ts"]).groupby(["id"])["y_clust_avg"].shift(1)
    X["next_clust"] = X["next_clust"].fillna(X["y_clust_avg"])
    X["prev_clust"] = X["prev_clust"].fillna(X["y_clust_avg"])
    X["slope_clust"] = ((X["y_clust_avg"] - X["prev_clust"]) + ((X["next_clust"] - X["prev_clust"]) / 2)) / 2
    
    ## ratio of fitted demand to cluster average
    X["ratio_hw_to_avg"] = X["y_hw"] / X["y_clust_avg"]
    X["ratio_hw_to_avg"] = np.nan_to_num(X["ratio_hw_to_avg"], nan=0, posinf=0, neginf=0)
    lag_cols = [x for x in X.columns if ("y_lag_" in x) and ("div" not in x)]
    for c in lag_cols:
        col = "ratio_hw_to_{}".format(c.replace("y_", ""))
        X[col] = X["y_hw"] / X[c]
        X[col] = np.nan_to_num(X[col], nan=0, posinf=0, neginf=0)
        
        col = "ratio_clust_to_{}".format(c.replace("y_", ""))
        X[col] = X["y_clust_avg"] / X[c]
        X[col] = np.nan_to_num(X[col], nan=0, posinf=0, neginf=0)
    
    ## cluster features
    X = pd.merge(
        X, cluster_features.rename(columns={"yhat": "y_clust_avg_norm"}), 
        on=["id", "ts"], how="left"
    )
    print("joined cluster feature")
    print(X.id.unique())
    #print(X.groupby('id')['ts'].min())
    X["next_clust_norm"] = X[["ts", "id", "y_clust_avg_norm"]].sort_values(by=["id", "ts"]).groupby(["id"])["y_clust_avg_norm"].shift(-1)
    X["prev_clust_norm"] = X[["ts", "id", "y_clust_avg_norm"]].sort_values(by=["id", "ts"]).groupby(["id"])["y_clust_avg_norm"].shift(1)
    X["next_clust_norm"] = X["next_clust_norm"].fillna(X["y_clust_avg_norm"])
    X["prev_clust_norm"] = X["prev_clust_norm"].fillna(X["y_clust_avg_norm"])
    X["slope_clust_norm"] = ((X["y_clust_avg_norm"] - X["prev_clust_norm"]) + ((X["next_clust_norm"] - X["prev_clust_norm"]) / 2)) / 2
    
    
    X["id"] = X["id"].astype(int)
#     X["cluster"] = X["cluster"].astype(int)
    
    train_dates = dates["train"]
    test_dates = dates["test"]
    X.to_csv("op/all_x.csv")
    train = X.loc[X.ts.isin(train_dates)].reset_index(drop=True)
    test = X.loc[X.ts.isin(test_dates)].reset_index(drop=True)
    
    all_ids_in_train = train.id.unique()
    test = test[test['id'].isin(all_ids_in_train)]
    
    print(X.shape, train.shape, test.shape)
    
    x_train = train.drop(columns=["y", "ts", "sku", "month", "woy"],errors='ignore')
    x_test = test.drop(columns=["y", "ts", "sku", "month", "woy"],errors='ignore')
    y_train = train["y"]
    y_test = test["y"]
    
    return train, test, x_train, x_test, y_train, y_test 

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor

### Modeling

In [ ]:
df.info()

In [ ]:
models_all = {}
for this_version in range(len(versions)):
    print("Version {}".format(versions[this_version]))
    this_df = df[df['ts'] < np.min(all_dates[versions[this_version]]['forecast'])]
    pred_hw = pred_hws[versions[this_version]]
    y_sliding = y_sliding_all[versions[this_version]]
    cluster_features_actual = cluster_features_actual_all[versions[this_version]]
    cluster_features = cluster_features_all[versions[this_version]]
    
    models = {}
    drop_cols = []
    
    train, test, x_train, x_test, y_train, y_test = process_data(
        df.drop(columns=["sku_description"]), all_dates[versions[this_version]],
        pred_hw, y_sliding[2], cluster_features_actual, cluster_features,
    )

    x_train.drop(columns=drop_cols, inplace=True)
    x_test.drop(columns=drop_cols, inplace=True)

    x_train.dropna(axis=1, inplace=True)
    x_test = x_test[x_train.columns]

    # x_train.fillna(0, inplace=True)
    # x_test.fillna(0, inplace=True)

    cols = x_train.select_dtypes(exclude=np.number).columns.tolist() + ["sku", "prod_code", "cases", "size"]
    encoders = {}
    for c in cols:
        if c not in x_train.columns:
            continue
        if c == "id":
            continue
        enc = LabelEncoder()
        if "event" in c:
            enc.fit(pd.concat([x_train[c], x_test[c]]))
            x_train[c] = enc.transform(x_train[c])
            x_test[c] = enc.transform(x_test[c])
        else:
            x_train[c] = enc.fit_transform(x_train[c])
            x_test[c] = enc.transform(x_test[c])
        encoders.update({(versions[this_version], c): enc})
    model = RandomForestRegressor(
        n_estimators=500,
        max_features="sqrt",
        n_jobs=-1,
        random_state=42
    )
    model.fit(x_train, y_train)

    yhat_train = model.predict(x_train)
    yhat_test = model.predict(x_test)

    yhat_train = np.clip(yhat_train, 0, None)
    yhat_test = np.clip(yhat_test, 0, None)

    models = {
        "version" : versions[this_version],
        "model": model,
        "train": train,
        "test": test,
        "x_train": x_train,
        "x_test": x_test,
        "y_train": y_train,
        "y_test": y_test,
        "yhat_train": yhat_train,
        "yhat_test": yhat_test,
    }
    models_all.update({versions[this_version]: models})

In [ ]:
#models_all = {}
for this_version in range(len(versions)):
    models = models_all[versions[this_version]]
    k = models["model"]
    x_train = models["x_train"]

    x_train.dropna(axis=1, inplace=True)

    fea_imp = pd.DataFrame({
        "imp": k.feature_importances_, 
        "fea": x_train.columns
    })
    fea_imp.sort_values(by="imp", ascending=True, inplace=True)
    fea_imp.set_index("fea", inplace=True)
    fea_imp.tail(30).plot.barh(figsize=(7,10))
    plt.title("Version {}".format(versions[this_version]))
    plt.show()        

In [ ]:
rf_yhat = []
for this_version in range(len(versions)):
    models = models_all[versions[this_version]]
    aaa = models["train"].copy()
    aaa["yhat"] = models["yhat_train"]
    aaa['version'] = versions[this_version]
    this_rf_yhat = [aaa]

    aaa = models["test"].copy()
    aaa["yhat"] = models["yhat_test"]
    this_rf_yhat.append(aaa)
    this_rf_yhat = pd.concat(this_rf_yhat).reset_index(drop=True)
    this_rf_yhat['version'] = versions[this_version]
    rf_yhat.append(this_rf_yhat)

rf_yhat = pd.concat(rf_yhat).reset_index(drop=True)

In [ ]:
for this_version in range(len(versions)):
    this_rf_yhat = rf_yhat[rf_yhat['version'] == versions[this_version]]
    ts = this_rf_yhat.groupby("ts").sum()
    plt.figure(figsize=(15,5))
    plt.plot(ts.y, label="Actuals")
    plt.plot(ts.yhat, label="Forecast main")
    plt.plot(ts.y_hw, label="Forecast Holts")
    plt.plot(ts.y_clust_avg, label="Forecast Cluster Avg")
    plt.legend(loc = "upper right")
    plt.show()

In [ ]:
for this_version in range(len(versions)):
    for c in rf_yhat.id.unique():
        ts = rf_yhat.loc[ (rf_yhat.id == c) & (rf_yhat.version == versions[this_version])].groupby("ts").sum()
        plt.figure(figsize=(15,5))
        plt.plot(ts.y, label="Actuals")
        plt.plot(ts.yhat, label="Forecast main")
        plt.plot(ts.y_hw, label="Forecast Holts")
        plt.legend(loc = "upper right")
        plt.title(" ID: {} Version : {}".format(c,versions[this_version] ))
        plt.show()

### Results

In [ ]:
results = pd.DataFrame()
for i in range(len(versions)):
    this_ts = rf_yhat[rf_yhat["version"] == versions[i]]
    for _id in this_ts.id.unique():
        ts = this_ts.loc[(this_ts["id"] == _id)].reset_index(drop=True)
        test = ts.loc[ ts.ts.isin( all_dates[versions[i]]["test"]) ].copy()
        
        metrics = calc_metrics(test.y, test.yhat)
        metrics.update({
            "id": _id,
            "version" : versions[i],
            "sku": ts["sku"][0],
            "customer": ts["customer"][0],
            "test_start": all_dates[versions[i]]["test"][0],
            "test_end": all_dates[versions[i]]["test"][-1],
            "total_actual": test.y.sum(),
            "total_forecast": test.yhat.sum(),
        })
        results = results.append(metrics, ignore_index=True, sort=False)

main_cols = ["id", "sku", "customer", "version", "test_start", "test_end", "total_actual", "total_forecast"]
results = results[main_cols + list(results.drop(columns=main_cols).columns)].copy()
results["sp1"] = results["total_actual"] / results["total_forecast"]
results["sp1"] = np.nan_to_num(results["sp1"], nan=1, posinf=0, neginf=0)


results.sort_values(by="id").reset_index(drop=True).set_index(["version", "id"])
results.groupby("customer").mean()
results.groupby("customer").median()

for i in range(len(versions)): 
    this_results = results[results['version'] == versions[i]]
    plt.figure(figsize=(15,5))
    plt.ylim(0,3)
    sns.boxplot(data=this_results, y="sp1", x="customer")
    plt.title("Version {}".format(versions[i]))
    plt.show()

    

## Permutation importance

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
models_all = {}
fea_imps_all = {}
for this_version in range(len(versions)):
    print("Version {}".format(versions[this_version]))
    this_df = df[df['ts'] < np.min(all_dates[versions[this_version]]['forecast'])]
    pred_hw = pred_hws[versions[this_version]]
    y_sliding = y_sliding_all[versions[this_version]]
    cluster_features_actual = cluster_features_actual_all[versions[this_version]]
    cluster_features = cluster_features_all[versions[this_version]]
    
    models = {}
    drop_cols = []
    fea_imps = {}
    
    train, test, x_train, x_test, y_train, y_test = process_data(
        df.drop(columns=["sku_description"]), all_dates[versions[this_version]],
        pred_hw, y_sliding[2], cluster_features_actual, cluster_features,
    )

    x_train.drop(columns=drop_cols, inplace=True)
    x_test.drop(columns=drop_cols, inplace=True)

    x_train.dropna(axis=1, inplace=True)
    x_test = x_test[x_train.columns]

    # x_train.fillna(0, inplace=True)
    # x_test.fillna(0, inplace=True)

    cols = x_train.select_dtypes(exclude=np.number).columns.tolist() + ["sku", "prod_code", "cases", "size"]
    encoders = {}
    for c in cols:
        if c not in x_train.columns:
            continue
        if c == "id":
            continue
        enc = LabelEncoder()
        if "event" in c:
            enc.fit(pd.concat([x_train[c], x_test[c]]))
            x_train[c] = enc.transform(x_train[c])
            x_test[c] = enc.transform(x_test[c])
        else:
            x_train[c] = enc.fit_transform(x_train[c])
            x_test[c] = enc.transform(x_test[c])
        encoders.update({(versions[this_version], c): enc})
    model = RandomForestRegressor(
        n_estimators=500,
        max_features="sqrt",
        n_jobs=-1,
        random_state=42
    )
    model.fit(x_train, y_train)

    r = permutation_importance(
        model, 
    #   x_train, y_train,
        x_test, y_test,
        n_repeats=30,
        random_state=123,
    )

    fea_imps = pd.DataFrame({
        "imp": r.importances_mean, 
        "fea": x_train.columns,
        "version":versions[this_version]
    })
    fea_imps.sort_values(by="imp", ascending=True, inplace=True)
    fea_imps.set_index("fea", inplace=True)
    fea_imps.tail(50).plot.barh(figsize=(7,10))
    plt.show()
    fea_imps_all.update({versions[this_version]: fea_imps})


In [ ]:
fea_imps_all[8].to_csv("fea_imp_rf.csv")

In [ ]:
fea_imps_all

## Refit without negatives

In [ ]:
models_all = {}
for this_version in range(len(versions)):
    fea_imps = fea_imps_all[versions[this_version]]
    fea_imps.drop(columns={'version'},inplace=True)
    thres = np.quantile(fea_imps.imp, 0.1)
    thres = 0
    print(thres)
    fea_imps.loc[fea_imps.imp < thres].shape, fea_imps.loc[fea_imps.imp >= thres].shape
    drop_cols = fea_imps.loc[(fea_imps.imp < thres)].index.values
    print(drop_cols)
    print("Version {}".format(versions[this_version]))
    this_df = df[df['ts'] < np.min(all_dates[versions[this_version]]['forecast'])]
    pred_hw = pred_hws[versions[this_version]]
    y_sliding = y_sliding_all[versions[this_version]]
    cluster_features_actual = cluster_features_actual_all[versions[this_version]]
    cluster_features = cluster_features_all[versions[this_version]]
    
    models = {}
    train, test, x_train, x_test, y_train, y_test = process_data(
        df.drop(columns=["sku_description"]), all_dates[versions[this_version]],
        pred_hw, y_sliding[2], cluster_features_actual, cluster_features,
    )

    x_train.drop(columns=drop_cols, inplace=True)
    x_test.drop(columns=drop_cols, inplace=True)

    x_train.dropna(axis=1, inplace=True)
    x_test = x_test[x_train.columns]

    # x_train.fillna(0, inplace=True)
    # x_test.fillna(0, inplace=True)

    cols = x_train.select_dtypes(exclude=np.number).columns.tolist() + ["sku", "prod_code", "cases", "size"]
    encoders = {}
    for c in cols:
        if c not in x_train.columns:
            continue
        if c == "id":
            continue
        enc = LabelEncoder()
        if "event" in c:
            enc.fit(pd.concat([x_train[c], x_test[c]]))
            x_train[c] = enc.transform(x_train[c])
            x_test[c] = enc.transform(x_test[c])
        else:
            x_train[c] = enc.fit_transform(x_train[c])
            x_test[c] = enc.transform(x_test[c])
        encoders.update({(versions[this_version], c): enc})
    model = RandomForestRegressor(
        n_estimators=500,
        max_features="sqrt",
        n_jobs=-1,
        random_state=42
    )
    model.fit(x_train, y_train)

    yhat_train = model.predict(x_train)
    yhat_test = model.predict(x_test)

    yhat_train = np.clip(yhat_train, 0, None)
    yhat_test = np.clip(yhat_test, 0, None)

    models = {
        "version" : versions[this_version],
        "model": model,
        "train": train,
        "test": test,
        "x_train": x_train,
        "x_test": x_test,
        "y_train": y_train,
        "y_test": y_test,
        "yhat_train": yhat_train,
        "yhat_test": yhat_test,
    }
    models_all.update({versions[this_version]: models})

In [ ]:


#models_all = {}
for this_version in range(len(versions)):
    models = models_all[versions[this_version]]
    k = models["model"]
    x_train = models["x_train"]

    x_train.dropna(axis=1, inplace=True)

    fea_imp = pd.DataFrame({
        "imp": k.feature_importances_, 
        "fea": x_train.columns
    })
    fea_imp.sort_values(by="imp", ascending=True, inplace=True)
    fea_imp.set_index("fea", inplace=True)
    fea_imp.tail(30).plot.barh(figsize=(7,10))
    plt.title("Version {}".format(versions[this_version]))
    plt.show()        

In [ ]:
rf_yhat = []
for this_version in range(len(versions)):
    models = models_all[versions[this_version]]
    aaa = models["train"].copy()
    aaa["yhat"] = models["yhat_train"]
    aaa['version'] = versions[this_version]
    this_rf_yhat = [aaa]

    aaa = models["test"].copy()
    aaa["yhat"] = models["yhat_test"]
    this_rf_yhat.append(aaa)
    this_rf_yhat = pd.concat(this_rf_yhat).reset_index(drop=True)
    this_rf_yhat['version'] = versions[this_version]
    rf_yhat.append(this_rf_yhat)

rf_yhat = pd.concat(rf_yhat).reset_index(drop=True)

In [ ]:
for this_version in range(len(versions)):
    this_rf_yhat = rf_yhat[rf_yhat['version'] == versions[this_version]]
    ts = this_rf_yhat.groupby("ts").sum()
    plt.figure(figsize=(15,5))
    plt.plot(ts.y, label="Actuals")
    plt.plot(ts.yhat, label="Forecast main")
    plt.plot(ts.y_hw, label="Forecast Holts")
    plt.plot(ts.y_clust_avg, label="Forecast Cluster Avg")
    plt.legend(loc = "upper right")
    plt.show()

In [ ]:
for this_version in range(len(versions)):
    for c in rf_yhat.id.unique():
        ts = rf_yhat.loc[ (rf_yhat.id == c) & (rf_yhat.version == versions[this_version])].groupby("ts").sum()
        plt.figure(figsize=(15,5))
        plt.plot(ts.y, label="Actuals")
        plt.plot(ts.yhat, label="Forecast main")
        plt.plot(ts.y_hw, label="Forecast Holts")
        plt.legend(loc = "upper right")
        plt.title(" ID: {} Version : {}".format(c,versions[this_version] ))
        plt.show()

### Results

In [ ]:
results = pd.DataFrame()
for i in range(len(versions)):
    this_ts = rf_yhat[rf_yhat["version"] == versions[i]]
    for _id in this_ts.id.unique():
        ts = this_ts.loc[(this_ts["id"] == _id)].reset_index(drop=True)
        test = ts.loc[ ts.ts.isin( all_dates[versions[i]]["test"]) ].copy()
        
        metrics = calc_metrics(test.y, test.yhat)
        metrics.update({
            "id": _id,
            "version" : versions[i],
            "sku": ts["sku"][0],
            "customer": ts["customer"][0],
            "test_start": all_dates[versions[i]]["test"][0],
            "test_end": all_dates[versions[i]]["test"][-1],
            "total_actual": test.y.sum(),
            "total_forecast": test.yhat.sum(),
        })
        results = results.append(metrics, ignore_index=True, sort=False)

main_cols = ["id", "sku", "customer", "version", "test_start", "test_end", "total_actual", "total_forecast"]
results = results[main_cols + list(results.drop(columns=main_cols).columns)].copy()
results["sp1"] = results["total_actual"] / results["total_forecast"]
results["sp1"] = np.nan_to_num(results["sp1"], nan=1, posinf=0, neginf=0)


results.sort_values(by="id").reset_index(drop=True).set_index(["version", "id"])
results.groupby("customer").mean()
results.groupby("customer").median()

for i in range(len(versions)): 
    this_results = results[results['version'] == versions[i]]
    plt.figure(figsize=(5,2))
    plt.ylim(0,3)
    sns.boxplot(data=this_results, y="sp1", x="customer")
    plt.title("Version {}".format(versions[i]))
    plt.show()

    

In [ ]:

plt.figure(figsize=(15,5))
plt.ylim(0,3)
sns.boxplot(data=results, y="sp1", x="customer")
plt.show()


In [ ]:
results.mean(), results.median()

In [ ]:
# a = pd.merge(att[["id", "sku_code"]], rf_yhat, on=["id"], how="right")
# print(a.shape, rf_yhat.shape)
# a.to_csv("prediction_sellout_rf_v1_0819.csv", index=False)
# a.to_parquet("prediction_sellout_rf_v1_0819.parquet")